<a href="https://colab.research.google.com/github/Huang-Yongzhi/musiclm-pytorch/blob/main/musiclm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install musiclm-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 125.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 125.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 

# Usage
`MuLaN` first needs to be trained

In [23]:
import torch
from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer

audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 128,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer
)

# get a ton of <sound, text> pairs and train

wavs = torch.randn(2, 1024)
texts = torch.randint(0, 20000, (2, 256))

loss = mulan(wavs, texts)
loss.backward()

# after much training, you can embed sounds and text into a joint embedding space
# for conditioning the audio LM

embeds = mulan.get_audio_latents(wavs)  # during training

embeds = mulan.get_text_latents(texts)  # during inference

spectrogram yielded shape of (65, 86), but had to be cropped to (64, 80) to be patchified for transformer


To obtain the conditioning embeddings for the three transformers that are a part of AudioLM, you must use the `MuLaNEmbedQuantizer` as so

In [24]:
from musiclm_pytorch import MuLaNEmbedQuantizer

# setup the quantizer with the namespaced conditioning embeddings, unique per quantizer as well as namespace (per transformer)

quantizer = MuLaNEmbedQuantizer(
    mulan = mulan,                          # pass in trained mulan from above
    conditioning_dims = (1024, 1024, 1024), # say all three transformers have model dimensions of 1024
    namespaces = ('semantic', 'coarse', 'fine')
)

# now say you want the conditioning embeddings for semantic transformer

wavs = torch.randn(2, 1024)
conds = quantizer(wavs = wavs, namespace = 'semantic') # (2, 8, 1024) - 8 is number of quantizers

In [25]:
!pip install audiolm_pytorch

# 加载数据集

1.数据集内容

调用的.csv文件内容如下
```
ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval
-0Gj8-vB1q4,30,40,"/m/0140xf,/m/02cjck,/m/04rlf","['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']","The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.",4,False,True
...
```

**解释**：
数据集是一个包含音频信息和描述的元数据文件，格式类似于 CSV。每行包含一个 YouTube 音频的标识符（ytid），音频的开始和结束时间（start_s 和 end_s），音频标签（audioset_positive_labels）和其他相关信息。

使用如 youtube-dl 这类工具来下载视频，然后使用音频处理库（例如 librosa 或 pydub）来裁剪音频。以下是一个大致的步骤指南：

# 1. 使用 youtube-dl
下载 YouTube 音频
首先，您需要安装 youtube-dl。在 Colab 中，您可以使用以下命令安装：

In [8]:
# !pip install youtube-dl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.6 MB/s eta 0:00:00


In [13]:
# !pip install --upgrade youtube-dl


In [22]:
# import librosa
# import soundfile as sf
# import os
# import pandas as pd
# from youtube_dl import YoutubeDL

# def trim_audio(file_path, start_time, end_time, output_path):
#     y, sr = librosa.load(file_path, sr=None, offset=start_time, duration=end_time - start_time)
#     sf.write(output_path, y, sr)

# def download_youtube_audio(ytid, output_dir):
#     ydl_opts = {
#         'format': 'bestaudio/best',
#         'postprocessors': [{
#             'key': 'FFmpegExtractAudio',
#             'preferredcodec': 'wav',
#             'preferredquality': '192',

#         }],
#         'verbose': True,
#         'outtmpl': os.path.join(output_dir, '%(id)s.%(ext)s')
#     }

#     try:
#         with YoutubeDL(ydl_opts) as ydl:
#             ydl.download([f'http://www.youtube.com/watch?v={ytid}'])
#     except Exception as e:
#         print(f"Error downloading video {ytid}: {e}")
#         return False  # Indicate failure
#     return True  # Indicate success


# # 加载CSV文件
# csv_file = 'musiccaps-public.csv'
# df = pd.read_csv(csv_file)

# # 遍历CSV文件，下载并裁剪音频
# for index, row in df.iterrows():
#     ytid = row['ytid']
#     start_s = row['start_s']
#     end_s = row['end_s']
#     if download_youtube_audio(ytid, 'downloaded_audios'):
#         try:
#             trim_audio(f'downloaded_audios/{ytid}.wav', start_s, end_s, f'trimmed_audios/{ytid}.wav')
#         except Exception as e:
#             print(f"Error trimming audio for video {ytid}: {e}")

[debug] Encodings: locale UTF-8, fs utf-8, out UTF-8, pref UTF-8
[debug] youtube-dl version 2021.12.17
[debug] Python version 3.10.12 (CPython) - Linux-5.15.120+-x86_64-with-glibc2.35
[debug] exe versions: ffmpeg 4.4.2, ffprobe 4.4.2
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}


[youtube] -0Gj8-vB1q4: Downloading webpage


ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 815, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 836, in __extract_info
    ie_result = ie.extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/common.py", line 534, in extract
    ie_result = self._real_extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/youtube.py", line 1794, in _real_extract
    'uploader_id': self._search_regex(r'/(?:channel|user)/([^/?&#]+)', owner_profile_url, 'uploader id') if owner_profile_url else None,
  File "/usr/local/lib/python3.10/dist-pa

Error downloading video -0Gj8-vB1q4: ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
[youtube] -0SdAVK79lg: Downloading webpage


ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 815, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 836, in __extract_info
    ie_result = ie.extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/common.py", line 534, in extract
    ie_result = self._real_extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/youtube.py", line 1794, in _real_extract
    'uploader_id': self._search_regex(r'/(?:channel|user)/([^/?&#]+)', owner_profile_url, 'uploader id') if owner_profile_url else None,
  File "/usr/local/lib/python3.10/dist-pa

...

ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 815, in wrapper
    return func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/YoutubeDL.py", line 836, in __extract_info
    ie_result = ie.extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/common.py", line 534, in extract
    ie_result = self._real_extract(url)
  File "/usr/local/lib/python3.10/dist-packages/youtube_dl/extractor/youtube.py", line 1794, in _real_extract
    'uploader_id': self._search_regex(r'/(?:channel|user)/([^/?&#]+)', owner_profile_url, 'uploader id') if owner_profile_url else None,
  File "/usr/local/lib/python3.10/dist-pa

Error downloading video 0abzD7hBTRk: ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


[debug] exe versions: ffmpeg 4.4.2, ffprobe 4.4.2
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}


[youtube] 0bP2MH3LqvI: Downloading webpage


KeyboardInterrupt: ignored

## 使用Youtube-dl会报错
改用you-get

In [3]:
!pip install you-get

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 4.7 MB/s eta 0:00:00


测试一下You-Get

In [27]:
# !you-get -i 'https://www.youtube.com/watch?v=jNQXAC9IVRw'

site:                YouTube
title:               Me at the zoo
streams:             # Available quality and codecs
    [ DASH ] ____________________________________
    - itag:          133
      container:     mp4
      quality:       320x240 (240p)
      size:          0.7 MiB (741996 bytes)
    # download-with: you-get --itag=133 [URL]

    - itag:          395
      container:     mp4
      quality:       320x240 (240p)
      size:          0.5 MiB (567669 bytes)
    # download-with: you-get --itag=395 [URL]

    - itag:          242
      container:     webm
      quality:       320x240 (240p)
      size:          0.5 MiB (545262 bytes)
    # download-with: you-get --itag=242 [URL]

    - itag:          160
      container:     mp4
      quality:       192x144 (144p)
      size:          0.5 MiB (504329 bytes)
    # download-with: you-get --itag=160 [URL]

    - itag:          394
      container:     mp4
      quality:       192x144 (144p)
      size:          0.4 MiB (445711 by

检查可用格式：运行 you-get 命令带 -i 选项（用于信息查看模式），查看该视频支持的所有可用格式。这样可以帮助您了解是否有特定的音频格式可供下载。执行命令如下：

In [32]:
!you-get -i "https://www.youtube.com/watch?v=-0Gj8-vB1q4"


site:                YouTube
title:               lds music - perfect love
streams:             # Available quality and codecs
    [ DASH ] ____________________________________
    - itag:          244
      container:     webm
      quality:       640x480 (480p)
      size:          7.6 MiB (7975173 bytes)
    # download-with: you-get --itag=244 [URL]

    - itag:          397
      container:     mp4
      quality:       640x480 (480p)
      size:          6.4 MiB (6670720 bytes)
    # download-with: you-get --itag=397 [URL]

    - itag:          243
      container:     webm
      quality:       480x360 (360p)
      size:          6.1 MiB (6354867 bytes)
    # download-with: you-get --itag=243 [URL]

    - itag:          396
      container:     mp4
      quality:       480x360 (360p)
      size:          5.5 MiB (5723034 bytes)
    # download-with: you-get --itag=396 [URL]

    - itag:          135
      container:     mp4
      quality:       640x480 (480p)
      size:          5.

In [5]:
!you-get --no-caption -o "./downloaded_videos" --itag=160 "https://www.youtube.com/watch?v=-0Gj8-vB1q4"


site:                YouTube
title:               lds music - perfect love
stream:
    - itag:          160
      container:     mp4
      quality:       192x144 (144p)
      size:          4.0 MiB (4186336 bytes)
    # download-with: you-get --itag=160 [URL]

you-get: [error] oops, something went wrong.
you-get: don't panic, c'est la vie. please try the following steps:
you-get:   (1) Rule out any network problem.
you-get:   (2) Make sure you-get is up-to-date.
you-get:   (3) Check if the issue is already known, on
you-get:         https://github.com/soimort/you-get/wiki/Known-Bugs
you-get:         https://github.com/soimort/you-get/issues
you-get:   (4) Run the command with '--debug' option,
you-get:       and report this issue with the full output.


安装**ffmpeg** 或其他类似工具来从下载的视频文件中提取音频。

In [7]:
!sudo apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [21]:
import subprocess
import librosa
import soundfile as sf
import os
import pandas as pd
import glob # 用于文件路径名的模式匹配
from datetime import datetime

# def trim_audio(file_path, start_time, end_time, output_path):
#     y, sr = librosa.load(file_path, sr=None, offset=start_time, duration=end_time - start_time)
#     sf.write(output_path, y, sr)




# def download_lowest_resolution_video(ytid, video_output_dir):
#     video_url = f'https://www.youtube.com/watch?v={ytid}'
#     try:
#         # 使用 you-get 下载分辨率最低的视频
#         subprocess.run(['you-get', '--no-caption', '-o', video_output_dir, '--itag=160', video_url], check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Error downloading video {ytid}: {e}")
#         return False  # Indicate failure
#     return True  # Indicate success


def get_latest_file_in_dir(directory):
    """ 获取指定目录中最新的文件 """
    list_of_files = glob.glob(os.path.join(directory, '*'))
    if not list_of_files:  # 如果目录为空
        return None
    latest_file = max(list_of_files, key=os.path.getmtime)
    return latest_file



def download_lowest_resolution_video(ytid, video_output_dir):
    video_url = f'https://www.youtube.com/watch?v={ytid}'
    try:
      # 使用 you-get 下载分辨率最低的视频
        subprocess.run(['you-get', '--no-caption', '-o', video_output_dir, '--itag=160', video_url], check=True)
        print (f"Download {ytid} video.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading video {ytid}: {e}")
        return None
    # 查找下载的视频文件
    return get_latest_file_in_dir(video_output_dir)



def extract_audio_from_video(video_path, output_audio_path):
    try:
        result = subprocess.run(['ffmpeg', '-i', video_path, '-vn', '-acodec', 'aac', output_audio_path], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        file_name = video_path.split('/')[-1]
        print (f"Extract {file_name} video.")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting audio from video {video_path}: {e}\nOutput: {e.stdout.decode()}\nError: {e.stderr.decode()}")





# 加载CSV文件
csv_file = 'musiccaps-public.csv'
df = pd.read_csv(csv_file)

video_output_dir = './downloaded_videos'
audio_output_dir = './downloaded_audios'

# 确保输出目录存在
os.makedirs(video_output_dir, exist_ok=True)
os.makedirs(audio_output_dir, exist_ok=True)

test_n = 0
# 遍历CSV文件，下载视频并提取音频
for index, row in df.iterrows():
  if test_n >= 3:
    break
  ytid = row['ytid']
  downloaded_video = download_lowest_resolution_video(ytid, video_output_dir) # 下载视频
  if downloaded_video:
    # video_path = os.path.join(video_output_dir, f'{ytid}.mp4')  # 假设视频文件扩展名为 .mp4，命名可能不成功
    audio_path = os.path.join(audio_output_dir, f'{ytid}.m4a')   # 输出音频文件为 .m4a
    extract_audio_from_video(downloaded_video, audio_path)
  test_n += 1

Download -0Gj8-vB1q4 video.
Extract lds music - perfect love.mp4 video.
Download -0SdAVK79lg video.
Extract Learn an Easy Country Blues Lead Guitar Lesson - EP144.mp4 video.
Download -0vPFx-wRRI video.
Extract 徳永ゆうき - 「涙そうそう～指ぱっちん編～」.mp4 video.


下载文件内容

hubert_base_ls960.pt 文件是一个预训练的模型权重文件，用于 **HuBERT （Hidden Unit BERT）模型**。HuBERT 是由Facebook AI 研究团队开发的一种**自监督学习的语音识别模型**。它是基于 BERT 架构的，专门针对语音处理任务进行了优化。

In [26]:
import requests

def download_file(url, filename):
    response = requests.get(url)
    response.raise_for_status()  # 检查请求是否成功

    with open(filename, 'wb') as f:
        f.write(response.content)

# 设置文件的URL和你想要保存的文件名
file_url = "https://dl.fbaipublicfiles.com/hubert/hubert_base_ls960.pt"
file_name = "hubert_base_ls960.pt"

# 下载文件
download_file(file_url, file_name)

# 设置文件的URL和你想要保存的文件名
file_url = "https://dl.fbaipublicfiles.com/hubert/hubert_base_ls960_L9_km500.bin"
file_name = "hubert_base_ls960_L9_km500.bin"

# 下载文件
download_file(file_url, file_name)



SemanticTransformerTrainer（这可能是一个音频处理或自然语言处理相关的训练器）

In [6]:
# 不是我们要的音频链接
# import requests

# url = "https://github.com/hsfzxjy/models.storage/releases/download/HRNet-OCR/hrnet_cs_8090_torch11.pth"
# response = requests.get(url)
# response.raise_for_status()

# file_name = url.split('/')[-1]

# with open(file_name, 'wb') as f:
#   f.write(response.content)






To train (or finetune) the three transformers that are a part of `AudioLM`, you simply follow the instructions over at `audiolm-pytorch` for training, but pass in the `MulanEmbedQuantizer` instance to the training classes under the keyword `audio_conditioner`

ex. `SemanticTransformerTrainer`

In [27]:
import torch
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer

wav2vec = HubertWithKmeans(
    checkpoint_path = 'hubert_base_ls960.pt',
    kmeans_path = 'hubert_base_ls960_L9_km500.bin'
)


semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6,
    audio_text_condition = True      # this must be set to True (same for CoarseTransformer and FineTransformers)
).cuda()

trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder ='/content/downloaded_audios',
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1
)

trainer.train()

RuntimeError: ignored

In [ ]:
# you need the trained AudioLM (audio_lm) from above
# with the MulanEmbedQuantizer (mulan_embed_quantizer)

from musiclm_pytorch import MusicLM

musiclm = MusicLM(
    audio_lm = audio_lm,                 # `AudioLM` from https://github.com/lucidrains/audiolm-pytorch
    mulan_embed_quantizer = quantizer    # the `MuLaNEmbedQuantizer` from above
)

music = musiclm('the crystalline sounds of the piano in a ballroom', num_samples = 4) # sample 4 and pick the top match with mulan